## Equalizadores

Um equalizador é um dispositivo que trabalha diretamente com as frequências. Diferentemente dos filtros passa-baixa, passa-alta, passa-faixa e rejeita-faixa, que reduzem a amplitude do espectro de áudio acima ou abaixo de uma determinada frequência de corte, o equalizador molda o espectro de áudio ao realçar certas faixas de frequência enquanto deixa outras faixas inalteradas.

Normalmente, o objetivo de um equalizador é melhorar a qualidade sonora em sistemas de reprodução de som, bem como aumentar o ganho acústico no caso de sistemas de reforço sonoro Antes do uso generalizado da tecnologia digital, era comum a utilização de equalizadores analógicos em telefones fixos utilizados para trunking. Nessa seção iremos demonstrar dois dos filtros equalizadores mais utilizados no mercado: o filtro shelving e o filtro peak.
<center><p>
<img src= "https://homestudiobrasil.com.br/wp-content/uploads/2017/05/Screen-Shot-2017-05-16-at-5.10.33-PM.png" width=400px>
</p>
<p>
    <em>Imagem de https://homestudiobrasil.com.br/2017/05/16/equalizadores-entendendo-seu-funcionamento/</em>
</p></center>


Importaremos todas as funções apresentadas no capítulo de introdução mais o módulo **math** próprio do Python para utilizar as funções matemáticas tangente e cosseno e a constante matemática π.

Low Shelving - Leva junto todas as frequências abaixo da escolhida. É bastante utilizado quando o resultado que se pretende vai além de uma frequência ou região específica e pretende-se ressaltar ou atenuar todos os graves proporcionalmente.

High Shelving - Leva junto todas as frequências abaixo da escolhida. É bastante utilizado quando o resultado que se pretende vai além de uma frequência ou região específica e pretende-se ressaltar ou atenuar todos os agudos proporcionalmente.

paramétrico (peak) - Neste equalizador pode-se escolher a freqüência central a ser manipulada, a largura de banda (Q-bandwidth = quantidade de freqüências que se altera junto com a freqüência central) e a amplitude desta banda (ganho/atenuação).

<em>Fonte: https://www.avmakers.com.br/blog/equalizador-o-que-e-e-qual-a-sua-utilidade </em>

In [28]:
%run -i 1introducao.ipynb

from math import tan, pi, cos

In [29]:
def run_filter(cutoff_frequency_int, file_name, filter_function, gain, filter_class = 'lowpass', center_frequency = 0, bandwidth = 0):
    # leitura do arquivo WAV
    sample_rate, input_signal = read_audio_file(file_name)
 
    # aplicação do teorema de nyquist e normalização das frequências de corte
    nyq = 0.5 * sample_rate
    normal_cutoff = cutoff_frequency_int / nyq
    center_frequency = center_frequency / nyq
    bandwidth = bandwidth / nyq

    # aplicação do filtro shelving ou peak
    if filter_function == 'shelving':
        filtered_data, w, h = shelving(input_signal, normal_cutoff, gain, filter_class)
    else:
        filtered_data, w, h = peak(input_signal, center_frequency, bandwidth, gain)
  
    # plotagem da resposta em frequência do filtro
    plot_filter(cutoff_frequency_int, sample_rate, w, h)
   
    # escrita do áudio de saída em arquivo
    write_audio_file(filtered_data, sample_rate, 'output_file_equalizadores.wav')

    # plotagem dos sinais de entrada e saída no domínio do tempo e da frequência
    %matplotlib ipympl
    plot_signal(input_signal, sample_rate, 'Sinal de Entrada')
    plot_signal(filtered_data, sample_rate, 'Sinal de Saída')
 
    # mostra os reprodutores de áudio do sinal de entrada e de saída
    display_audio(file_name, 'output_file_equalizadores.wav')


In [30]:

def shelving(input_signal, normal_cutoff, gain, filter_class):
    # inicialização do vetor de saída
    output_signal = [0] * len(input_signal)

    # paramêtros das eq. diferenciais 
    V0 = 10**(gain/20)
    H0 = V0 - 1

    # boost
    if gain >= 0:
        c = (tan(pi*normal_cutoff/2) - 1)/(tan(pi*normal_cutoff/2)+1)
    # cut
    else:
        c = (tan(pi*normal_cutoff/2) - V0)/(tan(pi*normal_cutoff/2)+V0)

    xh = 0

    # aplicação de eq. diferencias do filtro
    for n in range(len(input_signal)):
        xh_new = input_signal[n] - c*xh
        ap_y = c * xh_new + xh
        xh = xh_new
        if filter_class == 'lowpass':
            output_signal[n] = 0.5 * H0 * (input_signal[n] + ap_y) + input_signal[n]
        else:
            output_signal[n] = 0.5 * H0 * (input_signal[n] - ap_y) + input_signal[n]

    # calcula a resposta em frequência do filtro.
    w, h = freqz(output_signal, input_signal, fs=44100)

    return output_signal, w, h


def peak(input_signal, center_frequency, bandwidth, gain):
    # inicialização do vetor de saída
    output_signal = [0] * len(input_signal) 

    # paramêtros das eq. diferenciais: ganho
    V0 = 10**(gain/20)
    H0 = V0 - 1

    # boost
    if gain >= 0:
        c = (tan(pi*bandwidth/2) - 1)/(tan(pi*bandwidth/2)+1)
    # cut
    else:
        c = (tan(pi*bandwidth/2) - V0)/(tan(pi*bandwidth/2)+V0)

    # paramêtros das eq. diferenciais: centro de frequência
    d = -cos(pi*center_frequency)
    xh = [0,0]

    # aplicação de eq. diferencias do filtro
    for n in range(len(input_signal)):
        xh_new = input_signal[n] - d*(1-c)*xh[0] + c*xh[1]
        ap_y = -c * xh_new + d*(1-c)*xh[0] + xh[1]
        xh = [xh_new, xh[0]]
        output_signal[n] = 0.5 * H0 * (input_signal[n] - ap_y) + input_signal[n]

    # calcula a resposta em frequência do filtro
    w, h = freqz(output_signal, input_signal, fs=44100)

    # calculo do centro de frequência real do sinal a partir da largura de banda
    magnitude = np.abs(h)
    limiar = 1
    indices_superior = np.where(magnitude > limiar)[0]
    indices_inferior = np.where(magnitude[::-1] > limiar)[0]
    frequencia_superior = w[indices_superior[0]]
    frequencia_inferior = w[::-1][indices_inferior[0]]
    largura_banda = frequencia_superior - frequencia_inferior
    center = largura_banda/2
    print("Frequência central calculada:", abs(center))

    return output_signal, w, h


# reseta os widgets
clear_output()
out1.clear_output()
out2.clear_output()
out3.clear_output()

# roda o filtro e apresenta o widget
run_filter(5000, 'symphony.wav', 'shelving', 6, 'lowpass')
widgets_equalizers()

Dropdown(description='Classe do filtro:', options=('lowpass', 'highpass'), style=DescriptionStyle(description_…

Dropdown(description='Tipo do filtro:', options=('shelving', 'peak'), style=DescriptionStyle(description_width…

Text(value='symphony.wav', continuous_update=False, description='Nome do arquivo:', style=DescriptionStyle(des…

BoundedIntText(value=6, description='Ganho:', max=12, min=-12, style=DescriptionStyle(description_width='initi…

IntSlider(value=13000, continuous_update=False, description='Freq. de corte:', max=22000, min=1000, step=500, …

Button(button_style='success', description='Atualizar', style=ButtonStyle())

Output()